# Test CRAWDAD User

TJ Kim

Date: 4/20/21

#### Summary:
Import and run Crawdad users.

In [1]:
# Import Generic Classes
import numpy as np
import copy
import random

# Import All Custom Classes
import os, sys
sys.path.append(os.path.pardir+"/solvers")
sys.path.append(os.path.pardir+"/one_mobility")
sys.path.append(os.path.pardir+"/crawdad_mobility")


from Server import *
from User import *
from Link import *
from Job import *
from get_one_sim_usr import *
from load_crawdad_usr import *

# Import Solver Classes
from Optim_PlanGenerator import *
from SeqGreedy_PlanGenerator import*
from Migration_Plans import *

In [2]:
# Load All 12 users and trim their info
db_names = []
for i in range(1,13):
    db_names += ['LifeMap_GS'+str(i)+'.db']
    
db = []
for name in db_names:
    name = '/home/tkim/CMU/MEC_research/crawdad_mobility/' + name
    df,trace = load_usr(name)
    db += [trace]

In [3]:
"""
Make Simulation Parameters
"""
sim_param = Sim_Params(time_steps=15, x_length = 7, y_length = 11, max_edge_length=3,num_path_limit=10)
boundaries = np.array([[-sim_param.x_length,sim_param.x_length],[sim_param.y_length,sim_param.y_length]])

"""
Make Servers
"""

# Server Settings
num_server_l1 = 2
num_server_l2 = 6
num_server_l3 = 1

num_resource = 3
weak_range = np.array([[4,8],[1000,1500],[4,16]])
strong_range = np.array([[50,100],[100000,150000],[300,600]])

rsrc_cost = np.array([0.03, 0.01, 0.05])

rsrc_cost_scale_lv1 = 2
rsrc_cost_scale_lv2 = 1
rsrc_cost_scale_lv3 = 0.7

# Generate Server
servers_l1 = []
servers_l2 = []
servers_l3 = []
idx_counter = 0

for i in range(num_server_l1):
    servers_l1.append(Server(boundaries,level=1,rand_locs=True,locs=None))
    servers_l1[-1].server_resources(num_resource, weak_range, strong_range)
    servers_l1[-1].assign_id(idx_counter)
    servers_l1[-1].server_resources_cost(num_resource,rsrc_cost*rsrc_cost_scale_lv1)
    idx_counter += 1

for i in range(num_server_l2):
    servers_l2.append(Server(boundaries,level=2,rand_locs=True,locs=None))
    servers_l2[-1].server_resources(num_resource, weak_range, strong_range)
    servers_l2[-1].assign_id(idx_counter)
    servers_l2[-1].server_resources_cost(num_resource,rsrc_cost*rsrc_cost_scale_lv2)
    idx_counter += 1

for i in range(num_server_l3):
    servers_l3.append(Server(boundaries,level=3,rand_locs=False,locs=np.array([200,200])))
    servers_l3[-1].server_resources(num_resource, weak_range, strong_range)
    servers_l3[-1].assign_id(idx_counter)
    servers_l3[-1].server_resources_cost(num_resource,rsrc_cost*rsrc_cost_scale_lv3)
    idx_counter += 1

servers = servers_l1 + servers_l2 + servers_l3


"""
Make Links
"""

# Link Settings
num_link = [0,1,2,3]
prob_link = [0.5,0.2,0.2,0.1]
lv_minmax = np.array(([[500,1000],[10000,20000],[30000,50000]]))
lv1_transmission = 1
link_costs = np.array([0.05, 0.02, 0.01])
latency_settings = [10, 1] #[ms per switch, ms per mile]

links = Link(servers, num_link, prob_link, lv_minmax, link_costs, latency_settings,lv1_transmission)


In [56]:
"""
Make Users 
- First draw samples from dataste - 
"""

# User Settings
num_user = 1

max_speed = 80
num_path = 5
num_path_orig = 1

# Generate Server
users = []
idx_counter = 0

for i in range(num_user):
    
    comfirmed = False
    
    while confirmed is False:
        db_idx = np.random.randint(len(db))
        mvmt_array = random.choice(list(db[db_idx].values()))
        print(mvmt_array.shape[0])
        if mvmt_array.shape[0] > sim_param.time_steps:
            comfirmed = True
        
    users += [Crawdad_User(boundaries, sim_param.time_steps, max_speed, num_path, num_path_orig, mvmt_array)]
    users[-1].generate_MC(servers)
    users[-1].assign_id(idx_counter)
    idx_counter += 1
    
    

